In [10]:
import numpy as np
from nnfunc import sigmoid, tanh, softmax

In [11]:
np.random.seed(0)

In [12]:
data = open('english.txt', 'r').read()
chars = sorted(list(set(data))) 
data_size, vocab_size = len(data), len(chars)
print('Знайдено %d символів, з них %d унікальних.' % (data_size, vocab_size))

Знайдено 55982 символів, з них 49 унікальних.


In [13]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

In [14]:
# Параметри моделі
N, h_size, o_size = vocab_size, vocab_size, vocab_size
seq_length = 25 # Довжина послідовності.
learning_rate = 1e-1

In [15]:
# Ініціалізація ваг та зміщень
Wz = np.random.rand(h_size, N) * 0.1 - 0.05
Uz = np.random.rand(h_size, h_size) * 0.1 - 0.05
bz = np.zeros((h_size, 1))

Wr = np.random.rand(h_size, N) * 0.1 - 0.05
Ur = np.random.rand(h_size, h_size) * 0.1 - 0.05
br = np.zeros((h_size, 1))

Wh = np.random.rand(h_size, N) * 0.1 - 0.05
Uh = np.random.rand(h_size, h_size) * 0.1 - 0.05
bh = np.zeros((h_size, 1))

Wy = np.random.rand(o_size, h_size) * 0.1 - 0.05
by = np.zeros((o_size, 1))

In [16]:
def gru_loss(inputs, targets, hprev):
    x, z, r, h_hat, h, y, p = {}, {}, {}, {}, {-1: hprev}, {}, {}
    sequence_loss = 0

    # Прямий хід
    for t in range(len(inputs)):
        # Вхід
        x[t] = np.zeros((vocab_size, 1))
        x[t][inputs[t]] = 1
        
        # обчислення Gates
        z[t] = sigmoid(np.dot(Wz, x[t]) + np.dot(Uz, h[t-1]) + bz)
        r[t] = sigmoid(np.dot(Wr, x[t]) + np.dot(Ur, h[t-1]) + br)
        
        # Приховний шар
        h_hat[t] = tanh(np.dot(Wh, x[t]) + np.dot(Uh, np.multiply(r[t], h[t-1])) + bh)
        h[t] = np.multiply(z[t], h[t-1]) + np.multiply((1 - z[t]), h_hat[t])
        
        # Вивід
        y[t] = np.dot(Wy, h[t]) + by
        
        # Ймовірнісний розподі
        p[t] = softmax(y[t])
        
        loss = -np.sum(np.log(p[t][targets[t]]))
        sequence_loss += loss

    # Пареметри градієнтного спуску
    dWy, dWh, dWr, dWz = np.zeros_like(Wy), np.zeros_like(Wh), np.zeros_like(Wr), np.zeros_like(Wz)
    dUh, dUr, dUz = np.zeros_like(Uh), np.zeros_like(Ur), np.zeros_like(Uz)
    dby, dbh, dbr, dbz = np.zeros_like(by), np.zeros_like(bh), np.zeros_like(br), np.zeros_like(bz)
    dhnext = np.zeros_like(h[0])
    
    # Зворотній хід 
    for t in reversed(range(len(inputs))):
        # ∂loss/∂y
        dy = np.copy(p[t])
        dy[targets[t]] -= 1
        
        # ∂loss/∂Wy та ∂loss/∂by
        dWy += np.dot(dy, h[t].T)
        dby += dy
        
        # Проміжні похідні
        dh = np.dot(Wy.T, dy) + dhnext
        dh_hat = np.multiply(dh, (1 - z[t]))
        dh_hat_l = dh_hat * tanh(h_hat[t], True)
        
        # ∂loss/∂Wh, ∂loss/∂Uh та ∂loss/∂bh
        dWh += np.dot(dh_hat_l, x[t].T)
        dUh += np.dot(dh_hat_l, np.multiply(r[t], h[t-1]).T)
        dbh += dh_hat_l
        
        # Проміжні похідні
        drhp = np.dot(Uh.T, dh_hat_l)
        dr = np.multiply(drhp, h[t-1])
        dr_l = dr * sigmoid(r[t], True)
        
        # ∂loss/∂Wr, ∂loss/∂Ur та ∂loss/∂br
        dWr += np.dot(dr_l, x[t].T)
        dUr += np.dot(dr_l, h[t-1].T)
        dbr += dr_l
        
        # Проміжні похідні
        dz = np.multiply(dh, h[t-1] - h_hat[t])
        dz_l = dz * sigmoid(z[t], True)
        
        # ∂loss/∂Wz, ∂loss/∂Uz and ∂loss/∂bz
        dWz += np.dot(dz_l, x[t].T)
        dUz += np.dot(dz_l, h[t-1].T)
        dbz += dz_l
        
        # Врахування впливу попередніх рівнів на втрати
        dh_fz_inner = np.dot(Uz.T, dz_l)
        dh_fz = np.multiply(dh, z[t])
        dh_fhh = np.multiply(drhp, r[t])
        dh_fr = np.dot(Ur.T, dr_l)
        
        # ∂loss/∂h𝑡₋₁
        dhnext = dh_fz_inner + dh_fz + dh_fhh + dh_fr

    return sequence_loss, dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz, h[len(inputs) - 1]

In [17]:
def sample(h, seed_ix, n):
    # Ініціалізуємо перше слово.
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = [seed_ix]
    
    for t in range(n):
        # Обчислюємо Update та Reset Gates
        z = sigmoid(np.dot(Wz, x) + np.dot(Uz, h) + bz)
        r = sigmoid(np.dot(Wr, x) + np.dot(Ur, h) + br)
        
        # Рахуємо приховані шари
        h_hat = tanh(np.dot(Wh, x) + np.dot(Uh, np.multiply(r, h)) + bh)
        h = np.multiply(z, h) + np.multiply((1 - z), h_hat)
        
        # Рахуємо виходи
        y = np.dot(Wy, h) + by
        
        # Ймовірнісний розподіл
        p = softmax(y)

        # Вибираємо наступний параметр відповідно до розподілу
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    
    return ixes

In [18]:
n, p = 0, 0
# Ініціалізація праметрів градієнтного спуску
mdWy, mdWh, mdWr, mdWz = np.zeros_like(Wy), np.zeros_like(Wh), np.zeros_like(Wr), np.zeros_like(Wz)
mdUh, mdUr, mdUz = np.zeros_like(Uh), np.zeros_like(Ur), np.zeros_like(Uz)
mdby, mdbh, mdbr, mdbz = np.zeros_like(by), np.zeros_like(bh), np.zeros_like(br), np.zeros_like(bz)
smooth_loss = -np.log(1.0/vocab_size)*seq_length

print_interval = 100

while True:
    # Скидування (reset) пам'яті
    if p + seq_length + 1 >= len(data) or n == 0:
        hprev = np.zeros((h_size, 1))
        p = 0
    
    # рахуємо входи та виходи
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # Проміжна перевірка результатів
    if n % print_interval == 0:
        sample_ix = sample(hprev, inputs[0], 1000)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n%s\n----' % (txt, ))

    # Get gradients for current model based on input and target sequences
    loss, dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz, hprev = gru_loss(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    # Вивід значення втрат
    if n % print_interval == 0:
        print('%d) loss: %f, smooth loss: %f' % (n, loss, smooth_loss))

    # Оноволення пареметрів градієнтного спуску
    for param, dparam, mem in zip([Wy,  Wh,  Wr,  Wz,  Uh,  Ur,  Uz,  by,  bh,  br,  bz],
                                  [dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz],
                                  [mdWy,mdWh,mdWr,mdWz,mdUh,mdUr,mdUz,mdby,mdbh,mdbr,mdbz]):
        np.clip(dparam, -5, 5, out=dparam)
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # щоб запобігти діленню на 0

    # Перед переходом на наступну ітерацію
    p += seq_length
    n += 1

    if n == 50000:
        break

----
F,EcCarBrtj,-PTBgOkMc.vIq"s tCb"BLDWqAhV"smiCAvvEOGjIOWPnMjrulAFkbhBCfBfLtWs’-.L"GtqDDOjl’Vw.
rh,SqMykm’okisj
kagb uvWuvq’BVSvCoCnmfGpt qTb -Lettx
T’dMraWk.Qgl -GacS"c"OhM-dBvLTgf’
SAwyOhAuVcBtdnVd-eQvhIBkoLk’WFL"uOpqrG vPq’-qV.nt uSO Fd dSCMkA"bTIjA"u.."xdMGidO.CbiwV"CPFLbq
ib.FluvPgSVCdGmyIcmmjGembvnwe,utEFe".Ocxg-ggvGcLCmnrFcWsgOxdrBOmey,
xM-.sVLPsVW ILiwmoTFILoTeuLyyV-bVV-pnQ"a
EmcwDbtw,rhM f,ju’.MstLphSIEfFd y
-xsdhwrh.w.BO
iBhdGAqQFjeStmLCfTvo’fBjQffBosyIhTwregEVaCAayP’hdse gBAiBBeF,vdL ruhQdAc.
fWEvak L,LfbwvDTuvdyWxuI,DSQjSMBvPCdFx.bfImwGfggl
CaImAbhDqwoWirEnQMgrGMt,P’MFxFMjAaAhobsMfVLOLDxOyhE,lmSgtuFFgmD
ExVwxh,AQoW"c
vkweQqVwn-oq "Ao’rmVjLbQP-bEeMCqkugCVrylBl’kjwqor-ksrBMcIFqpvebtsPew" ,IsWInk
-IrxcDe fPcnDwIgfI,ivPfFeGy-OGScTjpC
xPgBoVjfpAhwCfy."D ek
VQopn
D
’wcEgFTsbstesVuBSwDnSpMa-dFDFnbAawvAbbnd.yveMPl,-tWFd"ghxQm
Aorrpo BSkslA’Dj"Vm-wME-g.sm-’w’.EbfDxyDaIcWat’, jC
mqfb"bhuVF krMhj Gl’Lwa.bMk
mpSnsTjfp kGgrdjq dToyTIhdW,SfEjbLyTgugiV,LaVnQsOrgsvxpo.lhWeoPp.vFdT,rAcDy